In [9]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error

In [10]:
def RMSE(y_test, y_pred):
    return mean_squared_error(y_test, y_pred, squared=False)

In [11]:
train_RMSE_list = []
test_RMSE_list = []

for i in range(10):

    train_data = pd.read_csv("../../data/BG LBStack/Train_Fold"+str(i)+"_lbstack.csv")
    test_data = pd.read_csv("../../data/BG LBStack/Test_Fold"+str(i)+"_lbstack.csv")
    
    train_naive = pd.read_csv("../../data/BG Naive/Train_Fold"+str(i)+".csv")
    train_naive["y12_shifted"] = train_naive.groupby("file_name")["y12"].shift(6)
    train_naive.dropna(inplace=True)
    
    test_naive = pd.read_csv("../../data/BG Naive/Test_Fold"+str(i)+".csv")
    test_naive["y12_shifted"] = test_naive.groupby("file_name")["y12"].shift(6)
    test_naive.dropna(inplace=True)
    
    train_x = train_data.drop("label", axis=1)
    train_y = train_data[["label"]]

    test_x = test_data.drop("label", axis=1)
    test_y = test_data[["label"]]
    
    train_RMSE = pd.DataFrame(
        train_x.apply(lambda x:RMSE(train_y.values.flatten(), x), axis=0),
        columns=["Fold"+str(i)]
    )

    train_RMSE_naive = pd.DataFrame(
        [RMSE(train_naive["y12"], train_naive["y12_shifted"])],
        columns=["Fold"+str(i)],
        index=["Naive"]
    )
    
    train_RMSE = train_RMSE.append(train_RMSE_naive)
    
    test_RMSE = pd.DataFrame(
        test_x.apply(lambda x:RMSE(test_y.values.flatten(), x), axis=0),
        columns=["Fold"+str(i)]        
    )

    test_RMSE_naive = pd.DataFrame(
        [RMSE(test_naive["y12"], test_naive["y12_shifted"])],
        columns=["Fold"+str(i)],
        index=["Naive"]
    )
    
    test_RMSE = test_RMSE.append(test_RMSE_naive)
    
    train_RMSE_list.append(train_RMSE)
    test_RMSE_list.append(test_RMSE)

train_RMSE_df = pd.DataFrame(np.concatenate(train_RMSE_list, axis=1),
                             index = train_RMSE.index,
                             columns = ["Fold"+str(i) for i in range(10)])
test_RMSE_df = pd.DataFrame(np.concatenate(test_RMSE_list, axis=1),
                            index = test_RMSE.index,
                            columns = ["Fold"+str(i) for i in range(10)])

In [12]:
import os
path = "../../data/BG LBStack Pred/"
if not os.path.exists(path):
    os.mkdir(path)

In [13]:
train_RMSE_df.to_csv("../../data/BG LBStack Pred/Train_RMSE_lbstack.csv")
test_RMSE_df.to_csv("../../data/BG LBStack Pred/Test_RMSE_lbstack.csv")